In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os, time, json, re
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import data_preprocessing as dp
import backtrader as bt
import matplotlib.pyplot as plt
import dask.dataframe as dd

## Get data

In [ ]:
frequency = timedelta(seconds=60)
pair = 'USDT_BTC'
date_start = '2021-10-02'
date_end = '2021-11-12'
lob_depth = 100
norm_type = 'dyn_z_score'
roll = 1440*10 # 10 days

In [ ]:
# df_data, df_data_stdz = dp.import_data(
#     pair, 
#     date_start, 
#     date_end, 
#     frequency=frequency, 
#     depth=lob_depth, 
#     norm_type=norm_type, 
#     roll=roll, 
#     stdz_depth=100
# )

In [ ]:
# results_trade = dp.get_trade_data(pair, date_start, date_end, frequency)

In [ ]:
results_px = dp.get_lob_data(pair, date_start, date_end, frequency, lob_depth)
df_px = dd.read_csv(results_px, compression='gzip').compute()

In [ ]:
df_px.plot(x='Datetime', y='Mid_Price', figsize=(12,4))

## Resample

In [ ]:
df_px['Datetime'] = pd.to_datetime(df_px['Datetime'])

# resample data to a less granular frequency
df_data = df_px.set_index('Datetime').asfreq('30min')
# df_data['volume'] = df_data['amount_buy'] + df_data['amount_sell']

data_resampled = df_data.resample('30min', label='right').agg( # closing time of candlestick
    {
    'Mid_Price': ['last', 'first', np.max, np.min], 
    # 'volume': np.sum
    }
)

data_resampled.columns = data_resampled.columns.get_level_values(1)

data_resampled['close'] = data_resampled['last']
data_resampled['open'] = data_resampled['first']
data_resampled['high'] = data_resampled['amax']
data_resampled['low'] = data_resampled['amin']
# data_resampled['volume'] = data_resampled['sum']
data_resampled.index.name = 'datetime'

data_resampled
# rename columns

In [ ]:
data_resampled['log_ret'] = (np.log(data_resampled['close']) - np.log(data_resampled['close'].shift(1)))
data_resampled['roll_std'] = data_resampled['log_ret'].rolling(window=336).std() # 336 is the number of 30mins interval in week
data_resampled['roll_std'].plot(figsize=(8,4))

## Backtrader

In [ ]:
from Strategies.GoldenCross import GoldenCross
from Strategies.BuyHold import BuyHold

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(GoldenCross)

# Create a Data Feed
data = bt.feeds.PandasData(dataname=data_resampled[:2000])

# Add the Data Feed to Cerebro
cerebro.adddata(data)

cerebro.addwriter(bt.WriterFile, out='./Strategies/logging/golden_cross2.csv', csv=True)

# Set our desired cash start
cerebro.broker.setcash(200000.0)
# Add a FixedSize sizer according to the stake
# cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
# cerebro.broker.setcommission(commission=0.0007) 

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything


cerebro.run()

plt.rcParams['figure.figsize']=[22, 16]
cerebro.plot()
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# figure out what's wrong with stop losses


In [ ]:
strategy_results = pd.read_csv('./Strategies/logging/golden_cross2.csv', header=1, index_col='Id').dropna(thresh=3)
strategy_results['datetime'] = pd.to_datetime(strategy_results['datetime'])
print(strategy_results.shape)

In [ ]:
strategy_and_indic = pd.merge(data_resampled, strategy_results, left_index=True, right_on='datetime', how='outer')
print(strategy_and_indic.columns)
columns_to_keep = ['datetime', 'open_x', 'close_x', 'high_x', 'low_x', 'cash', 'value', 'buy', 'sell', 'pnlplus', 'pnlminus', 'sma', 'sma.1', 'crossover']
strategy_and_indic[columns_to_keep].to_csv('./Strategies/logging/golden_cross_cl.csv')

In [ ]:
# def saveplots(cerebro, numfigs=1, iplot=True, start=None, end=None,
#              width=16, height=9, dpi=300, tight=True, use=None, file_path = '', **kwargs):

#         from backtrader import plot
#         if cerebro.p.oldsync:
#             plotter = plot.Plot_OldSync(**kwargs)
#         else:
#             plotter = plot.Plot(**kwargs)

#         figs = []
#         for stratlist in cerebro.runstrats:
#             for si, strat in enumerate(stratlist):
#                 rfig = plotter.plot(strat, figid=si * 100,
#                                     numfigs=numfigs, iplot=iplot,
#                                     start=start, end=end, use=use)
#                 figs.append(rfig)

#         for fig in figs:
#             for f in fig:
#                 f.savefig(file_path, bbox_inches='tight')
#         # return figs

# saveplots(cerebro, file_path = 'savefig.png') 

## My Strategy Backtester

In [ ]:
## Roadmap
# for each trade I need entry price, closing price, number of periods, time in the trade, min, max, volatility V 
# make execution assumptions: conservative: enter trade next open bar, exit trade next open bar V
# add stops and trailing stops - V static stops, TODO: trailing
# wrap strategy in a reusable class - V TODO: refinements and add trading metrics method
# pull more data, a few pairs and recent data (3 pairs, most recent data) - pending when pawel back, local storage
# add single strategy to bitstamp account with cctx V
# backtest multiple strategies across multiple pairs, splitting between train and test set etc
# deploy multiple strategies

## adjusted to accomodate for long only strategy without stop losses V
## add stop losses fixing any potential issue V

In [ ]:
# add trade profitability TODO: profitability dot not perfecty alligned
# 0 on stop loss period impacting returns
# probably need a refactoring to accomodate for multiple trades in the same period

In [ ]:

import ta
from ta.volatility import BollingerBands, AverageTrueRange
from ta.trend import EMAIndicator
import config
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from StratTest.engine import TradingStrategy
import itertools
import ccxt

import multiprocessing as mp

In [ ]:
frequency = timedelta(seconds=60)
pair = 'USDT_BTC'
date_start = '2020-11-11'
date_end = '2021-04-03'
lob_depth = 100

In [ ]:
results_px = dp.get_lob_data(pair, date_start, date_end, frequency, lob_depth)
df_px = dd.read_csv(results_px, compression='gzip').compute()

# prep
df_px['Datetime'] = pd.to_datetime(df_px['Datetime'])

# resample data to a less granular frequency - 
df_px = df_px.set_index('Datetime').asfreq('1min')

In [ ]:
frequencies = ('5min', '10min', '30min', '60min')
stop_losses = (100, 300, 500, 600, 1000)
mas_combinations = list(itertools.product(np.arange(5,105, 5), np.arange(10,260, 10), frequencies, stop_losses))
# mas_combinations = [(combo[0], combo[1], combo[2], combo[3]) for combo in mas_combinations if combo[0]<combo[1]]
# mas_combinations

In [ ]:
len(mas_combinations)

In [ ]:
def test_strategy(short_ema, long_ema, freq, stop_loss):
    
    trading_strategy = TradingStrategy(df_px, frequency=freq)
    trading_strategy.resample_data() # resampling


    trading_strategy.add_strategy(
        'EMACrossOverLO', 
        execution_type='current_bar_close',#'next_bar_open', 'current_bar_close, 'cheat_previous_close
        stop_loss_bps=stop_loss,
        comms_bps=50,
        short_ema=short_ema,
        long_ema=long_ema,
        print_trades=False
    )

    
    # strategy_performance.append()
    return [short_ema, long_ema, freq, stop_loss, trading_strategy.trades_df['cum_trades_pctg_return'][-1]]

In [ ]:
## Test different combinations of moving averages

frequencies = ('5min', '10min', '30min', '60min')
stop_losses = (100, 300, 500, 600, 1000)
mas_combinations = list(itertools.product(np.arange(5,105, 5), np.arange(10,260, 10), frequencies, stop_losses))
mas_combinations = [(combo[0], combo[1], combo[2], combo[3]) for combo in mas_combinations if combo[0]<combo[1]]

strategy_performance = []

for ma_combination, iteration in zip(mas_combinations, range(len(mas_combinations))):

    if iteration % 100 == 0: print(f'Done the first {iteration} iterations')

    short_ema = ma_combination[0]
    long_ema = ma_combination[1]
    freq = ma_combination[2]
    stop_loss = ma_combinations[3]

    result = test_strategy(short_ema, long_ema, freq, stop_loss)

    strategy_performance.append(result)

strategy_performance_df = pd.DataFrame(strategy_performance, columns=['short_ema', 'long_ema', 'frequency', 'stop_loss', 'performance']).sort_values('performance', ascending=False)
strategy_performance_df

In [ ]:
%%time

test_results = []
with mp.Pool(processes=8) as pool:
    results = pool.starmap(test_strategy, mas_combinations)
    test_results.append(results)

In [ ]:
strategy_performance_df = pd.DataFrame(results, columns=['short_ema', 'long_ema', 'frequency', 'stop_loss', 'performance']).sort_values('performance', ascending=False)
strategy_performance_df.head(40)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_predict, TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import seaborn as sns
import math

import matplotlib.pyplot as plt
import visualization_tools as viz_tools
# regressor = RandomForestRegressor(n_esti)

In [ ]:
trading_strategy = TradingStrategy(df_px, frequency='30min')
trading_strategy.resample_data() # resampling

df_rf_ = trading_strategy.df.copy()
df_rf_['y'] = (np.log(df_rf_['close']) - np.log(df_rf_['close'].shift(1))).shift(-1).copy()
df_rf_['y_cheat'] = (np.log(df_rf_['close']) - np.log(df_rf_['close'].shift(1))).shift(-1).copy()

df_rf_['open_close_var'] = ((df_rf_['close'] - df_rf_['open']) / df_rf_['close']).rolling(48*7).mean()
df_rf_['high_low_var'] = ((df_rf_['high'] - df_rf_['low']) / df_rf_['close']).rolling(48*7).mean()
df_rf_['log_return_1'] = (np.log(df_rf_['close']) - np.log(df_rf_['close'].shift(1)))
df_rf_['log_return_3'] = (np.log(df_rf_['close']) - np.log(df_rf_['close'].shift(1)))
df_rf_['log_return_5'] = (np.log(df_rf_['close']) - np.log(df_rf_['close'].shift(1)))

ema_indicator = EMAIndicator(df_rf_['close'], window=10)

df_rf_['ema_10'] = ema_indicator.ema_indicator()

ema_indicator = EMAIndicator(df_rf_['close'], window=20)

df_rf_['ema_20'] = ema_indicator.ema_indicator()

ema_indicator = EMAIndicator(df_rf_['close'], window=5)

df_rf_['ema_5'] = ema_indicator.ema_indicator()


df_rf_ = df_rf_.dropna()

In [ ]:
features = ['y_cheat','close', 'high', 'low', 'open', 'ema_10', 'ema_5', 'ema_20', 'log_return_1', 'log_return_3', 'open_close_var', 'high_low_var']

X = df_rf_[features].copy()
y = df_rf_[['y']].copy()

# initial split to keep out part of the data for validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=False, test_size=0.1, random_state=42)


test_size = 500
max_train_size = X_train.shape[0] - test_size

ts_cv = TimeSeriesSplit(
    n_splits=5,
    gap=48,
    # max_train_size=5000,
    test_size=test_size,
)

all_splits = list(ts_cv.split(X_train, y_train))

for split in all_splits:

    print(split[0][0], split[0][-1], split[1][0], split[1][-1])

# Access individual splits
# train_0, test_0 = all_splits[0]


In [ ]:
# X.iloc[train_0] # X.iloc[test_0]

In [ ]:
## TODO: decide whether a custom splitter to avoid leakage is needed
# gap = 48 # 1 day in 30m bars
# train_chunk_size = 1300+gap
# test_chunk_size = 200


# block_size = train_chunk_size + test_chunk_size

# blocks = [X[n:n+block_size] for n in np.arange(0, X.shape[0], block_size)]

# tr_test_blocks = [[block[gap:train_chunk_size], block[train_chunk_size:train_chunk_size+test_chunk_size]] for block in blocks]

# validation = tr_test_blocks.pop()

# len(tr_test_blocks)

In [ ]:
categorical_columns = [
    "weather",
    "season",
    "holiday",
    "workingday",
]
categories = [
    ["clear", "misty", "rain"],
    ["spring", "summer", "fall", "winter"],
    ["False", "True"],
    ["False", "True"],
]
ordinal_encoder = OrdinalEncoder(categories=categories)

# numerical_features = features = ['close', 'high', 'low', 'open', 'ema_10', 'ema_5', 'ema_20', 'log_return_5', 'open_close_var', 'high_low_var']


experimental_pipeline = make_pipeline(
    ColumnTransformer(
        transformers=[
            #("categorical", ordinal_encoder, categorical_columns),
        ],
        remainder="passthrough",
    ),
    # HistGradientBoostingRegressor(
    #     categorical_features=range(4),
    # ),
    RandomForestRegressor()

)

parameters = {
    # 'scaler': [StandardScaler(), MinMaxScaler(), Normalizer(), MaxAbsScaler()],
	# 'selector__threshold': [0, 0.001, 0.01],
	'randomforestregressor__n_estimators': [10, 20], #50, 100, 200],
	'randomforestregressor__max_depth': [1, 3, 5], #7, 11, 15, 20, 40],
	'randomforestregressor__min_samples_leaf': [1, 2, 4, 8, 16]
}

grid = GridSearchCV(
    experimental_pipeline, 
    parameters, 
    cv=all_splits, # pass an iterable
    #scoring='neg_mean_absolute_percentage_error',
    n_jobs=8, 
    verbose=1
).fit(X_train, y_train.values.ravel())
 
print('Training set score: ' + str(grid.score(X_train, y_train.values.ravel())))
print('Validation set score: ' + str(grid.score(X_valid, y_valid.values.ravel())))

In [ ]:
# Access the best set of parameters
best_params = grid.best_params_
print(best_params)
# Stores the optimum model in best_pipe
best_pipe = grid.best_estimator_
print(best_pipe)

In [ ]:
result_df = pd.DataFrame.from_dict(grid.cv_results_, orient='columns')
print(result_df.columns)
result_df.sort_values(by='rank_test_score')

In [ ]:
best_regressor = best_pipe.steps[1][1]

importances = best_regressor.feature_importances_
std = np.std([tree.feature_importances_ for tree in best_regressor.estimators_], axis=0) # check all trees inside best regressor ensamble

forest_importances = pd.Series(importances, index= X_train.columns)

fig, ax = plt.subplots()
forest_importances.plot.barh(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
sns.relplot(data=result_df,
            kind='line',
            x='param_randomforestregressor__n_estimators',
            y='mean_test_score',
            hue='param_randomforestregressor__min_samples_leaf',
            col='param_randomforestregressor__max_depth')
plt.show()

In [ ]:
# def evaluate(model, X, y, cv):
#     cv_results = cross_validate(
#         model,
#         X,
#         y,
#         cv=cv,
#         scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
#     )
#     mae = -cv_results["test_neg_mean_absolute_error"]
#     rmse = -cv_results["test_neg_root_mean_squared_error"]
#     print(
#         f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
#         f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
#     )


# evaluate(experimental_pipeline, X, y.values.ravel(), cv=ts_cv)

In [ ]:
pd.plotting.autocorrelation_plot(y_valid)
y_valid.plot()

In [ ]:
pd.plotting.autocorrelation_plot(y_train[:200])
y_train.plot()

In [ ]:
train_set_predictions = best_pipe.predict(X_train)
valid_predictions = best_pipe.predict(X_valid)

viz_tools.plot_timeseries(
    ts_list=[
        y_train, 
        pd.Series(train_set_predictions, index=y_train.index), 
        y_valid, 
        pd.DataFrame(valid_predictions, index=y_valid.index)
    ], 
    #primary_axis=[True],#,True,True,True], 
    legend=['y_training', 'sample_pred', 'y_validation', 'validation_pred'],
    sample_size=1
)

In [ ]:
train_set_predictions = best_pipe.predict(X_train)

fig, ax = plt.subplots()
ax.scatter(y_train, train_set_predictions, edgecolors=(0, 0, 0))
ax.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], "k--", lw=4)
ax.set_xlabel("Measured")
ax.set_ylabel("Predicted")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_valid, valid_predictions, edgecolors=(0, 0, 0))
ax.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], "k--", lw=4)
ax.set_xlabel("Measured")
ax.set_ylabel("Predicted")
plt.show()

In [ ]:
regressor = RandomForestRegressor(n_estimators=1000, oob_score=True, random_state=123, max_depth=10, n_jobs=8)
regressor.fit(X_train.values, y_train.values)
y_pred = regressor.predict(X_test.values)

In [ ]:
# train_error = mean_squared_error(y_train['y'], X_train)

# test_error = mean_squared_error(y_test['y'], X_test)

prediction_error = mean_squared_error(y_test['y'], y_pred)

print(prediction_error)

In [ ]:
importances = regressor.feature_importances_
pd.Series(importances, index=features)

In [ ]:
import plotly_express as px
fig = px.line()
fig.add_scatter(y=pd.Series(y_pred), name='pred')
fig.add_scatter(y=y_test['y'], name='real')
# px.line(pd.DataFrame(y_pred.T))
# px.line(pd.DataFrame(y_test))

In [ ]:
X_train.values

In [ ]:
fig = px.line()
y_sample_pred = regressor.predict(X_train.values)
fig.add_scatter(y=pd.Series(y_train['y'].values), name='real')
fig.add_scatter(y=y_sample_pred, name='sample_prediction')

fig

In [ ]:
# # testing different BB strategies
# frequencies = ('5min', '10min', '30min', '60min', '120min')
# window_devs = np.arange(0.4,4.1, 0.2)
# bb_combinations = list(itertools.product(np.arange(10,200, 5), window_devs, frequencies))
# strategy_performance = []
# for bb_combination in bb_combinations:

#     window = bb_combination[0]
#     window_dev = np.round(bb_combination[1], 3)
#     freq = bb_combination[2]
#     print(bb_combination)

#     trading_strategy = TradingStrategy(df_px, frequency=freq)
#     trading_strategy.resample_data() # resampling


#     trading_strategy.add_strategy(
#         'BollingerBandsLO', 
#         execution_type='current_bar_close',#'next_bar_open', 'current_bar_close, 'cheat_previous_close
#         stop_loss=0.0,
#         comms_bps=50,
#         window=window,
#         window_dev=window_dev,
#         print_trades=False
#     )


#     try: strategy_return = trading_strategy.trades_df['cum_trades_pctg_return'][-1]
#     except: strategy_return = np.nan
#     strategy_performance.append([bb_combination[0], bb_combination[1], bb_combination[2], strategy_return])

# strategy_performance_df_bb = pd.DataFrame(strategy_performance, columns=['bb_window', 'bb_window_devs', 'frequency', 'performance']).sort_values('performance', ascending=False)
# strategy_performance_df_bb

In [ ]:
trading_strategy.df['ma_diff'] = (trading_strategy.df['high'] - trading_strategy.df['high'])

In [ ]:
# TODO add BB and maybe another strategy for comparison
# implement capturing of data snapped - a simple database?
# make sure engine works as expected
# keep monitoring bot
# make sure work on grouper and performances and transaction cost holds
# why returns prices are different than close? slippage?


# NOTE on data gathered:
# orders are executed one bar after triggered. Whilst that's ok for for buy, is that fine for sells?
# issue with API not finding order executed sometimes: do we need to fetch all the time?
# no need to have stop loss price printed after closing position

# window = 45
# window_dev = 1.4


## Exchange connectivity
exchange = ccxt.bitstamp(
    {
        'apiKey': config.BITSTAMP_API_KEY,
        'secret': config.BITSTAMP_API_SECRET
    }
)

# bars = exchange.fetch_ohlcv('BTC/GBP', timeframe='30m', limit=1000)
# bars_df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
# bars_df['timestamp'] = pd.to_datetime(bars_df['timestamp'], unit='ms')
# bars_df.index = pd.DatetimeIndex(bars_df['timestamp'], freq='30min')

short_ema = 10
long_ema = 220
trading_strategy = TradingStrategy(df_px, frequency='30min') # df_px
trading_strategy.resample_data() # resampling
trading_strategy.df['bar_volatility'] = ((trading_strategy.df['high'] - trading_strategy.df['low']) / trading_strategy.df['close']).rolling(20).mean()
# trading_strategy.add_indicator('BollingerBands', window=20)



trading_strategy.add_strategy(
    'EMACrossOverLO', # EMACrossOverLO BollingerBandsLO
    execution_type='current_bar_close',#'next_bar_open', 'current_bar_close, 'cheat_previous_close
    stop_loss_bps=1000,
    comms_bps=25,
    short_ema=short_ema,
    long_ema=long_ema,
    print_trades=False
)



In [ ]:

trading_strategy.trading_chart(
    plot_strategy=True,
    plot_volatility=True,
    short_ema=f'ema_{short_ema}', 
    long_ema=f'ema_{long_ema}'
)

In [ ]:

trading_strategy.trading_chart(
    plot_strategy=True,
    plot_volatility=True,
    short_ema=f'ema_{short_ema}', 
    long_ema=f'ema_{long_ema}'
)

In [ ]:
dff = trading_strategy.df

In [ ]:
trading_strategy.trades_df

In [ ]:
import plotly_express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go



In [ ]:
trading_strategy.df

In [ ]:
trading_strategy.df['EMACrossOverLO_signal'].plot()

In [ ]:
trading_strategy.df.shift(1).head(5)

In [ ]:
trading_strategy.df['open_close_var'] = ((trading_strategy.df['close'] - trading_strategy.df['open']) / trading_strategy.df['close']).rolling(48*7).mean()
trading_strategy.df['high_low_var'] = ((trading_strategy.df['high'] - trading_strategy.df['low']) / trading_strategy.df['close']).rolling(48*7).mean()


fig = make_subplots(
    rows=2, 
    cols=1,
    shared_xaxes=True,
    row_heights=[0.5, 0.5],
    # vertical_spacing=0.02,
    specs=[
        [{"secondary_y": False}],
        [{"secondary_y": True}]
    ]
)


fig.add_scatter(x=trading_strategy.df.index, y=trading_strategy.df['high_low_var'], row=2, col=1, name='high_low')
fig.add_scatter(x=trading_strategy.df.index, y=trading_strategy.df['open_close_var'], secondary_y=True,row=2, col=1, name='open_close')
fig.add_scatter(x=trading_strategy.df.index, y=trading_strategy.df['close'], row=1, col=1, name='price')

fig.update_layout(height=1000)
fig

In [ ]:
((trading_strategy.df['open'] - trading_strategy.df['close']) / trading_strategy.df['close']).rolling(48*7).mean().plot()

In [ ]:
((trading_strategy.df['high'] - trading_strategy.df['low']) / trading_strategy.df['close']).rolling(48*7).mean().plot()

In [ ]:
trading_strategy.df['ma_diff'] = (trading_strategy.df['ema_20'] - trading_strategy.df['ema_110']) / (((trading_strategy.df['ema_20'] + trading_strategy.df['ema_110']))/2)
trading_strategy.df['ma_diff'].plot()

In [ ]:
trading_strategy = TradingStrategy(df_px, frequency='30min')
trading_strategy.resample_data()
trading_strategy.df

In [ ]:
trading_strategy = TradingStrategy(df_px, frequency='120min')
trading_strategy.resample_data() # resampling

short_ema = 20
long_ema = 30

trading_strategy.add_strategy(
    'EMACrossOverLO', 
    execution_type='current_bar_close',#'next_bar_open', 'current_bar_close, 'cheat_previous_close
    stop_loss=0.0,
    comms_bps=50,
    short_ema=short_ema,
    long_ema=long_ema,
    print_trades=False
)
trading_strategy.trading_chart(plot_strategy=True, short_ema=f'ema_{short_ema}', long_ema=f'ema_{long_ema}')

In [ ]:
trading_strategy.trades_df#['exit_price'].shift(1)

In [ ]:
trading_strategy.trades_df[:].head(10)


In [ ]:
trading_strategy.df.head(60)[['close','EMACrossOverLO_new_position', 'trade_grouper', 'px_returns_calcs', 'gross_log_returns', 'EMACrossOverLO_gross_log_returns']]

In [ ]:
past_trades = exchange.fetchMyTrades(pair)
open_orders = exchange.fetchOpenOrders(pair)
past_trades, open_orders

In [ ]:
trades_df = trading_strategy.df.groupby('trade_grouper').agg(
    entry_price=('execution_price', 'first'), 
    exit_price=('execution_price', 'last'), 
    trade_len=('trade_grouper', 'count'),
    direction=('EMACrossOverLO_new_position', 'first'),
    liquidated_at=('execution_time', 'last')
)

trades_df['trades_log_return'] = np.log(trades_df['exit_price']) - np.log(trades_df['entry_price'])
trades_df['cum_trades_log_return'] = trades_df['trades_log_return'].cumsum()

trades_df['trades_pctg_return'] = np.exp(trades_df['trades_log_return']) - 1
trades_df['cum_trades_pctg_return'] = np.exp(trades_df['cum_trades_log_return']) - 1

cum_return = f"{trades_df['cum_trades_pctg_return'][-1]:.2%}"

trades_df

In [ ]:
# add trade profitability TODO: profitability dot not perfecty alligned
# 0 on stop loss period impacting returns
# probably need a refactoring to accomodate for multiple trades in the same period
# proceede with more data

In [ ]:
# trading_strategy.df[(trading_strategy.df['EMACrossOver_new_position']!=0)|(trading_strategy.df['sl_hit']!=0)].head(20)

In [ ]:
trading_strategy.df.to_excel(f'StratTest/Exports/{trading_strategy.strategy}_{trading_strategy.stop_loss}.xlsx')

In [ ]:
# prepare df trades

# get positions in the dataframe where indicator generates signals
open_trades_idx = np.where(df['ema_cross_position']!=0)[0]
# -2 because of shape is n rows and df is 0 indexed and because we do + 1 later - avoid out of bound error
closing_trades_idx = np.append(open_trades_idx, df.shape[0]-2)[1:] 
df_trades = df.iloc[open_trades_idx][['ema_cross_position']].copy() # empty dataframe with only datetime index

# entry and closing points
df_trades['entry_price'] = df.iloc[open_trades_idx+1]['open'].values # assume entry trade is executed at the next bar open
df_trades['closing_price'] = df.iloc[closing_trades_idx+1]['open'].values # assume closing is executed at the next bar open

# trade discrete returns
df_trades['discrete_return'] = df_trades['ema_cross_position'] * ((df_trades['closing_price'] / df_trades['entry_price']) - 1)

# how long are the trades 
df_trades['trade_n_periods'] = closing_trades_idx - open_trades_idx
df_trades['trade_duration'] = df.iloc[closing_trades_idx].index - df.iloc[open_trades_idx].index

# what happened throughout the trade
df['trade_grouper'] = np.nan
df.loc[df.iloc[open_trades_idx].index, 'trade_grouper'] = df.iloc[open_trades_idx].index
df['trade_grouper'] = df['trade_grouper'].fillna(method='ffill')
df.head(60)

all_trades_list = []
for name, sub_df in df.groupby(by='trade_grouper'):
    max_val = sub_df['high'].max()
    min_val = sub_df['low'].min()
    returns_std = sub_df['returns'].std()

    all_trades_list.append([name, max_val, min_val, returns_std])


intra_trade_stats = pd.DataFrame(all_trades_list, columns=['datetime', 'px_high', 'px_low', 'returns_std']).set_index('datetime')
df_trades = pd.merge(df_trades, intra_trade_stats, left_index=True, right_index=True)


def max_dd_pctg(row):
    ''' Measure of how "painful" holding the trade was '''
    if row['ema_cross_position'] == 1:
        return (row['entry_price'] - row['px_low'])/row['px_low']
    elif row['ema_cross_position'] == -1:
        return (-(row['entry_price'] - row['px_high']))/row['px_high']
    else:
        return 0

df_trades['dd_pctg'] = df_trades.apply(max_dd_pctg, axis=1)

# calculate trade returns and jump into risk management / stop losses


In [ ]:

sl_trigger_time = sub_df[~(sub_df['sl_trigger'] < sub_df['low'])].index

# shortened trade time due to stop loss
stopped_sub_df = sub_df[sub_df.index<=sl_trigger_time[0]].copy()
stopped_sub_df['strategy_position'][-1] = -1

# remaining part of the trade, now position need to change to 0
quitted_sub_df = sub_df[sub_df.index>=sl_trigger_time[0]].copy()
quitted_sub_df

In [ ]:
sub_df.loc[sl_trigger_time, 'strategy_position'] = -1
sub_df


In [ ]:
sub_df[sub_df.index<=sl_trigger_time[0]]['strategy_position'][-1] = -1
sub_df[sub_df.index<=sl_trigger_time[0]]['strategy_position']

In [ ]:
def get_worst_price(row):
    ''' Get worst price relative to position '''
    if row['ema_cross_signal'] > 0:
        return min(row['open'], row['high'], row['low'], row['close'])
    elif row['ema_cross_signal'] < 0:
        return max(row['open'], row['high'], row['low'], row['close'])
    else:
        return 0



# static stop

sub_df['worst_price_timestamp'] = sub_df.apply(get_worst_price, axis=1)
# calculate loss vs worst price over the period
sub_df['cumulative_performance'] = sub_df['ema_cross_returns'].cumsum()
sub_df['worst_period_potential_loss'] = sub_df['ema_cross_signal'] * ((sub_df['worst_price_timestamp'] / entry_price) - 1)

sub_df[['ema_cross_returns', 'cumulative_performance', 'worst_period_potential_loss']]

In [ ]:
df_trades.apply(lambda x: (x['closing_price'] / x['entry_price']) - 1)

In [ ]:
## Metrics
# Net Profit
net_profit = df['ema_cross_cash'][-1] - initial_cash 

# Max Drowdown
max_dd = df_trades['dd_pctg'].max()

# Win Ratio
win_ratio = (df_trades['discrete_return']>0).sum() / df_trades.shape[0]

print(f'Net Profit: {net_profit:.2f}, Max Drawdown: {max_dd:.2%}, Win Ratio: {win_ratio:.2%}')

In [ ]:
# # df['close'].plot(legend=True)
# ((np.exp(df['ema_cross_returns'].cumsum()) * 100)).plot(legend=True)
# # ((np.exp(df['returns'].cumsum()) * df['close'][0])).plot(legend=True)

In [ ]:
# df['ema_cross_position'].cumsum().plot()
# df['ema_cross_signal'].plot()

In [ ]:
df[df.index>='2020-11-22 21:00:00'].head(50)[['trade_grouper', 'trade_grouper', 'close', 'low', 'high', 'sl_trigger', 'ema_cross_new_position', 'ema_cross_signal', 'ema_cross_trades', 'strategy_new_position', 'strategy_signal', 'strategy_trades']]

## Trading Bot

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ccxt
import schedule
from datetime import datetime
import config
import logging

import numpy as np
import pandas as pd

import StratTest.bot as bot

In [ ]:
# ## Exchange connectivity
# exchange = ccxt.binance(
#     {
#         'apiKey': config.BINANCE_API_KEY,
#         'secret': config.BINANCE_SECRET_KEY
#     }
# )

# markets = exchange.load_markets()
# pair = 'BTC/USD'

# bars = exchange.fetch_ohlcv(pair, timeframe='1m', limit=100) # most recent candle keeps evolving

# exchange.fetch_balance()

In [ ]:
pair = 'BTC/USDT'
strategy = 'EMACrossOver'
indicator = 'EMAIndicator'
short_ema = 10
long_ema = 20
my_bot = bot.TradingBot('EMACrossOverLO', indicator, sandbox=False, short_ema=short_ema, long_ema=long_ema)

In [ ]:
bars = my_bot.exchange.fetch_ohlcv('BTC/GBP', timeframe='30m', limit=100) # most recent candle keeps evolving
my_bot.bars_df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
my_bot.bars_df['timestamp'] = pd.to_datetime(my_bot.bars_df['timestamp'], unit='ms')
indicator_df = my_bot._get_crossover()

In [ ]:
my_bot.bars_df['ema_10'].plot()
my_bot.bars_df['ema_20'].plot()

In [ ]:
my_bot.exchange.has['cancelOrder']

In [ ]:
print(df_px.iloc[-1].to_dict())

In [ ]:
my_bot.bars_df.iloc[-1]

In [ ]:
symbol = 'BTC/GBP'

order_book = my_bot.exchange.fetchOrderBook(symbol)

ob_datetime = order_book['datetime']

top_ask_px = order_book['asks'][0][0]
top_ask_quantity = order_book['asks'][0][1]

top_bid_px = order_book['bids'][0][0]
top_bid_quantity = order_book['bids'][0][1]

top_mid_px = (top_ask_px + top_bid_px) / 2
top_ob_spread = (top_ask_px - top_bid_px) / top_mid_px

# current_mid_price : 1 BTC = my_size : x BTC
my_size_gbp = 22
trade_size = my_size_gbp/(top_mid_px)
trade_size, top_bid_px, top_ask_px, ob_datetime

In [ ]:
datetime.now().isoformat()

In [ ]:
pair = 'BTC/GBP'

In [ ]:
executed_orders = my_bot.exchange.fetchMyTrades(pair)
[order['order'] for order in executed_orders if order['order']=='1432301048147968']


In [ ]:
past_trades = exchange.fetchMyTrades(pair)
open_orders = exchange.fetchOpenOrders(pair)
past_trades, open_orders

In [ ]:
balances = my_bot.exchange.fetchBalance()
balances['GBP'], balances['BTC']

In [ ]:
my_bot.exchange.has['createLimitOrder']

In [ ]:
open_orders = my_bot.exchange.fetchOpenOrders('BTC/GBP') # open orders
settled_trades = my_bot.exchange.fetchMyTrades('BTC/GBP') # provides the history of settled trades


In [ ]:
my_bot.exchange.fetchBalance()['GBP']

## Reddit

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import config
import numpy as np
import pandas as pd
import time
import AlternativeData.reddit_sql as reddit_sql

from datetime import datetime

In [ ]:
from psaw import PushshiftAPI

api = PushshiftAPI()

In [ ]:
db_path = './AlternativeData/reddit_test.db'
reddit_sql.create_psaw_table(db_path)
reddit_sql.create_praw_tables(db_path)
reddit_sql.get_all_db_tables(db_path)

In [ ]:
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute('''DROP TABLE reddit_praw_submissions''')
c.execute('''DROP TABLE reddit_praw_comments''')
conn.close()

In [ ]:
start_datetime=datetime(2022, 1, 1, 0, 0, 0)
end_datetime=datetime(2022, 1, 16, 12, 0, 0)
freq='12H'
daterange = pd.date_range(start_datetime, end_datetime, freq=freq)

subreddits = ['wallstreetbets','cryptcurrency', 'bitcoin']

fields_list = ['id','author', 'created_utc', 'subreddit', 'title', 'selftext', 'full_link']

# TODO add retries
for subreddit in subreddits:

    for tstmp in daterange:

        print(f'Fetching {subreddit} data: start: {tstmp}, end: {tstmp + pd.Timedelta(freq)}')

        start_epoch = int(tstmp.timestamp())
        end_epoch = int((tstmp + pd.Timedelta(freq)).timestamp())

        submissions = api.search_submissions(
            after=start_epoch,
            before=end_epoch,
            subreddit=subreddit,
            filter=fields_list,
            limit=None
        )

        print('storing..')
        for submission in submissions:
                try:
                    reddit_sql.insert_new_psaw_submission(submission, db=db_path)
                except Exception as e:
                    print(e)
                    continue
        print('~~~ moving on ~~~')

In [ ]:
test_query_output = reddit_sql.get_reddits('reddit_psaw_submissions', db=db_path)
test_query_df = pd.DataFrame(test_query_output, columns=fields_list)

In [ ]:
test_query_df

In [ ]:
import praw

reddit = praw.Reddit(
    client_id=config.reddit_personal_use_script,
    client_secret=config.reddit_secret,
    # password=config.reddit_password,
    user_agent="rlt_bot",
    # username=config.reddit_username,
)

In [ ]:
comment.link_id

In [ ]:
counter

In [ ]:
reddit = praw.Reddit(
    client_id=config.reddit_personal_use_script,
    client_secret=config.reddit_secret,
    # password=config.reddit_password,
    user_agent="rlt_bot",
    # username=config.reddit_username,
)

In [ ]:
# submission = reddit.submission('eq4prj')
# for comment in submission.comments.list():
#     try:
#         print(comment.link_id)
#     except Exception as e:
#         print(e)
#         continue

In [ ]:
submission = reddit.submission(id=sub_id)

In [ ]:
submission = reddit.submission(id=sub_id)

reddit_sql.insert_new_praw_submission(submission, db=db_path)

In [ ]:
try: submission.body
except Exception as e: print(e)

In [ ]:
submission.comments.list()[415].deleted

In [ ]:
psaw_ids = pd.DataFrame(reddit_sql.get_reddits('reddit_psaw_submissions', db=db_path))[0]
praw_ids = pd.DataFrame(reddit_sql.get_reddits('reddit_praw_submissions', db=db_path))[0]
subs_list = psaw_ids[~psaw_ids.isin(praw_ids)].tolist()


# praw database update control flow
counter = 0

for sub_id in subs_list:

    try:
        if sub_id not in pd.DataFrame(reddit_sql.get_reddits('reddit_praw_submissions', db=db_path))[0].tolist():
            print(f'Working on {sub_id}')
            submission = reddit.submission(id=sub_id)

            reddit_sql.insert_new_praw_submission(submission, db=db_path)

            all_comments = submission.comments.list()
            print(f'number of comments: {len(all_comments)}')

            if len(all_comments)>0:
                # TODO some comments return no attributes, investigate whether
                # these are deleted comments or something else is going on
                for comment in all_comments:

                    reddit_sql.insert_new_praw_comment(comment, db=db_path)

    except Exception as e:
        print(f'{e} ### submission id: {sub_id}#') ### {submission.url}')
        continue


In [ ]:
comment_columns = ['praw_comment_id', 'praw_comment_parent_id', 'praw_comment_link_id', 'praw_comment_created_utc','praw_comment_body','praw_comment_score']
all_comments = pd.DataFrame(reddit_sql.get_reddits('reddit_praw_comments', db=db_path), columns=comment_columns)
print(all_comments.shape)
all_comments.head()

In [ ]:
all_comments.groupby('praw_comment_link_id')['praw_comment_id'].count().sort_values()

In [ ]:
all_comments[all_comments['praw_comment_link_id'] == 't3_eq4prj']

In [ ]:
submission.comments.list()

In [ ]:
submission.comments.list()[9].created_utc

In [ ]:
submission.comments.list()[9].link_id, submission.comments.list()[9].parent_id, submission.comments.list()[9].id

In [ ]:
submission.comments.list()[7].link_id, submission.comments.list()[7].parent_id, submission.comments.list()[7].id

In [ ]:
submission.comments.list()[4].link_id, submission.comments.list()[4].parent_id, submission.comments.list()[4].id

In [ ]:
submission.comments.list()[1].link_id, submission.comments.list()[1].parent_id, submission.comments.list()[1].id

In [ ]:
submission.comments.list()[9].parent_id

In [ ]:
submission.comments.list()[7].score

In [ ]:
# submission.comments.replace_more(limit=None)
for top_level_comment in submission.comments:
    print(top_level_comment.body)
    print()
    print('##')

In [ ]:
for top_level_comment in submission.comments:
    print(top_level_comment.body)

In [ ]:
data = pd.DataFrame(list(submissions))
data['created_utc'] = pd.to_datetime(data['created_utc'], unit='s')
data.head()

In [ ]:
data['id'].unique().shape

In [ ]:
db_name = 'reddit_testing.db'
reddit_sql.create_subreddit_table(db=f'./AlternativeData/{db_name}')

In [ ]:

data

In [ ]:
data['d_'].values

In [ ]:


# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(config.reddit_personal_use_script, config.reddit_secret)

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': config.reddit_username,
        'password': config.reddit_password}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'rlt_bot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

In [ ]:
# res = requests.get("https://oauth.reddit.com/r/python/hot",
#                    headers=headers)

# print(res.json())  # let's see what we get

# for post in res.json()['data']['children']:
#     print(post['data']['title'])

In [ ]:
def subreddit_df(res):

    df = pd.DataFrame()  # initialize dataframe

    # loop through each post retrieved from GET request
    for post in res.json()['data']['children']:
        # append relevant data to dataframe
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'author': post['data']['author'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': post['data']['created_utc'],
            'created': post['data']['created'],
            'edited': post['data']['edited'],
            'kind': post['kind'], # ie t3 - link
            'id': post['data']['id']
        }, ignore_index=True)

    df['created_utc'] = pd.to_datetime(df['created_utc'], utc=True, unit='s')
    df['created'] = pd.to_datetime(df['created'], utc=None, unit='s')
    # df['edited'] = pd.to_datetime(df['edited'], utc=None, unit='s') # 0 to be replaced with NAt

    # sort to make sure that oldest entry is at the bottom
    df = df.sort_values(by='created_utc', ascending=False) 
    return df


In [ ]:
subreddit = 'CryptoCurrency' # 'ETFs, CryptoCurrency
listing = 'new'
params = {'limit': 100}

final_df = pd.DataFrame()

# loop through 10 times (returning 1K posts)
for i in range(13):
    print(f'{i} iteration')
    try:
        # make a request for the trending posts in /r/[subreddit]
        res = requests.get(
            f"https://oauth.reddit.com/r/{subreddit}/{listing}",
            headers=headers,
            params=params
        )

        # get dataframe from response
        new_df = subreddit_df(res)
        new_df['bot_iteration'] = i

        # take the final row (oldest entry)
        row = new_df.iloc[new_df.shape[0]-1]
        # create fullname
        # fullname = row['kind'] + '_' + row['id']
        # print(fullname)
        
        # add/update fullname in params
        params['after'] = res.json()['data']['after']#fullname
        
        # append new_df to data
        final_df = final_df.append(new_df, ignore_index=True)

    except Exception as e:
        print(e)
        print(res.json())
        # time.sleep(60)
        continue

In [ ]:
final_df[final_df['id']=='s3orms']

In [ ]:
final_df.sort_values(by='created_utc')['id'].unique().shape

In [ ]:
# res = requests.get(
#     f"https://oauth.reddit.com/r/{subreddit}/{listing}",
#     headers=headers,
#     params=params
# )
# res.json()

In [ ]:
import sqlite3

In [ ]:
def insert_new_reddit(post):

    # using conn as a context manager to avoid explicitly committing every time
    with conn:
        values = {
            'author':post['author'],
            'id':post['id'],
            'created_utc':str(post['created_utc']),
            'subreddit':post['subreddit'],
            'title':post['title'],
            'selftext':post['selftext'],
            'ups':post['ups'],
            'downs':post['downs'],
            'upvote_ratio':post['upvote_ratio']
        }

        c.execute(f'''
            INSERT INTO reddit_test VALUES (
                :author,
                :id,
                :created_utc,
                :subreddit,
                :title,
                :selftext,
                :ups,
                :downs,
                :upvote_ratio
            )
        ''', values)


def get_reddits(ups_threshold):
    c.execute("SELECT * FROM reddit_test WHERE ups>=:ups", {'ups':ups_threshold})
    return c.fetchall()

In [ ]:
conn = sqlite3.connect(':memory:') # can pass a file 'filename.db' or make an in memory db - ':memory:' # './local_data/text_data.db'
c = conn.cursor() # create a cursor


c.execute('''
    CREATE TABLE reddit_test (
        author text,
        id text,
        created_utc text,
        subreddit text,
        title text,
        selftext text,
        ups integer,
        downs integer,
        upvote_ratio real
    )
''')
conn.commit()

# TODO use execute many instead for bulk upload
[insert_new_reddit(entry) for entry in final_df.to_dict('records')]

reddits = get_reddits(0)
columns = [d[0] for d in c.description]

conn.close()

In [ ]:
pd.DataFrame(reddits, columns=columns)